In [76]:
import cv2
import pykep
import numpy as np
import imutils

Image tools

In [4]:
def debris_in_pixels(physical_size, distance, alpha=np.pi / 6, beta=np.pi / 6, dimensions = (2464, 3280)):
    """
    Returns how many pixels the object at a given distance should take up
    :param physical_size: The size of the object
    :param distance: Distance between the target and observer
    :param alpha: horizontal FOV. Radians
    :param beta: vertical FOV. Radians
    :param dimensions: the pixel dimensions of the observer camera used (vertical, horisontal)
    """
    lengths = (2 * np.tan(alpha) * distance, 2 * np.tan(beta) * distance)  # vertical, horizontal
    px_per_length = dimensions[0] / lengths[0], dimensions[1] / lengths[1]
    new_shape = int(physical_size[0] * px_per_length[0]), int(physical_size[1] * px_per_length[1])
    return new_shape


def resize(original_image, physical_size, distance):
    """
    Resize the image based on the physical size of the target object and the relative distance between 
    target object and observer
    """
    width, height = debris_in_pixels(physical_size, distance)
    return cv2.resize(original_image, (width, height), interpolation=cv2.INTER_AREA)


def roi_in_frame(left_corner, object_size, frame):
    """
    Return the part of the ROI that is in the frame.
    :left_corner: the coordinates of the left corner of the object
    :object_size: size of the object in pixels in CV2 coordinates (columns, rows)
    """
    frame_size = frame.shape
    ROI = [[left_corner[0], left_corner[0] + object_size[0]],[left_corner[1], left_corner[1] + object_size[1]]]
    
    if left_corner[0] + object_size[0] > frame_size[0]:
        # outside the right bounding edge
        ROI[0] = [left_corner[0], frame_size[0]]
    if left_corner[0] < 0:
        # outside the left bounding edge
        ROI[0] = [0, left_corner[0] + object_size[0]]

    if left_corner[1] + object_size[1] > frame_size[1]:
        # outside the bottom bounding edge
        ROI[1] = [left_corner[1], frame_size[1]]
    if left_corner[1] < 0:
        # outside the upper bounding edge
        ROI[1] = [0, left_corner[1] + object_size[1]]
    
    return frame[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]]


def obj_in_frame(left_corner, frame, img):
    image_size = img.shape
    frame_size = frame.shape
    ROI = [[0,image_size[0]],[0,image_size[1]]]  # the whole image of the object
    if left_corner[0] + image_size[0] > frame_size[0]:
        # outside the right bounding edge
        ROI[0] = [0, frame_size[0] - left_corner[0]]
    if left_corner[0] < 0:
        # outside the left bounding edge
        ROI[0] = [-left_corner[0], image_size[0]]

    if left_corner[1] + image_size[1] > frame_size[1]:
        # outside the bottom bounding edge
        ROI[1] = [0, frame_size[1] - left_corner[1]]
    if left_corner[1] < 0:
        # outside the upper bounding edge
        ROI[1] = [-left_corner[1], image_size[1]]

    return img[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]]


def create_mask(img):
    return np.zeros(img.shape[:2], dtype="uint8")

Orbital Mechanics

In [ ]:
def propagate_based_on_vector(r,v, t):
    rf, vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    return np.array(rf), np.array(vf)

def propagate_based_on_elements(elements, t, kepler=True):
    r, v = pykep.par2ic(elements, mu=398600)
    rf,vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    if kepler == False:  # return vectors
        return np.array(rf),np.array(vf)
    if kepler == True:  # return kepler elements
        return pykep.ic2par(rf,vf,mu = 398600)

Utility

In [85]:
def delete_white_background(img, write=False):
    """
    Takes the images of Debris that I made in paint.
    Inverts the colours and deletes the white background.
    """
    img = np.invert(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, alpha = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    
    # Using cv2.split() to split channels 
    # of coloured image
    b, g, r = cv2.split(img)

    # Making list of Red, Green, Blue
    # Channels and alpha
    rgba = [b,g,r, alpha]

    # Using cv2.merge() to merge rgba
    # into a coloured/multi-channeled image
    dst = cv2.merge(rgba,3)

    # Writing and saving to a new image
    if write is True:
        cv2.imwrite("debris/deb.png", dst)
    return dst        

FOV

In [92]:
def to_body_axis(vector, r, v):
    """
    Calculates the rotation matrix and applies it. Returns the converted vector.
    """
    unit_r = r/np.linalg(r)
    unit_v = v/np.linalg(v)

    R = np.matrix([[unit_v[0], unit_r[1]*unit_v[2] - unit_v[1]*unit_r[2], unit_r[0]],
                [unit_v[1], unit_r[0]*unit_v[2] - unit_v[0]*unit_r[2], unit_r[1]],
                [unit_v[2], unit_r[0]*unit_v[1] - unit_v[0]*unit_r[1], unit_r[2]]])
    return np.matmul(R, vector)


In [ ]:
def in_front_of_Earth(r1, rel, R=6371):
    """
    Returns true if Earth is not in between the observer and target.
    """
    alpha = np.arctan(R/np.linalg.norm(r1))
    unit_r1 = r1/np.linalg.norm(r1)
    unit_rel = rel/np.linalg.norm(rel)
    beta = np.arccos(np.dot(unit_r1, -unit_rel))
    if beta <= alpha:
        return True
    if beta > alpha:
        return False

def in_FOV(r1, v1, r2, max_vertical=np.radians(60), max_horisontal=np.radians(60)):
    """
    Checks if the observer should be able to see the target.
    1 for observer. 2 for target.
    Returns True if yes
    """

    rel = r2 - r1

    if in_front_of_Earth(r1,rel) is False:
        return False
    
    rel_unit = (rel)/np.linalg.norm(rel)
    rel_unit = to_body_axis(rel_unit, r1, v1) 
    if rel_unit[2] < 0:
        return False

    horisontal_angle = abs(np.arctan(rel_unit[0]/rel_unit[2]))  # in the xz plane
    vertical_angle = abs(np.arctan(rel_unit[1]/rel_unit[2]))  # in yz plane
    if horisontal_angle <= max_horisontal and vertical_angle <= max_vertical:
        return True
    else:
        return False

Initialisation

In [90]:
elements_deb = [7290.2,0.0610,np.radians(30.379),np.radians(289.042),np.radians(293.776),np.radians(66.230)]
elements_sat = [7133.8,0.0043,np.radians(98.289),np.radians(49.866),np.radians(356.768),np.radians(149.512)]

In [7]:
vs = "Earth\sized_Earth_slowed.avi"
img_path = "debris\deb1.png"

In [89]:
object = cv2.imread(img_path)
object = delete_white_background(object, write=True)

(223, 260, 3)


(223, 260, 4)

In [ ]:
cap = cv2.VideoCapture(vs)  # open the video file
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('Earth_ball.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))
t = 0
if cap.isOpened() is False:
    print("Error opening video file")

Body

In [ ]:
while cap.isOpened():
    ret0, frame = cap.read()
    if ret0 is True:
        rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
        rT, vT = propagate_based_on_elements(elements_deb, t, kepler=False)
        